In [36]:
import pandas as pd
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.sql.types import *
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import seaborn as sns
from pyspark.sql.functions import col
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import Word2Vec
import matplotlib.pyplot as plt
from pyspark.sql.functions import split
from pyspark.sql.functions import expr
from machine_learning.testing import evaluateModel
from machine_learning.training import trainModel


ModuleNotFoundError: No module named 'machine_learning'

In [17]:
t = pd.read_json('Clothing_Shoes_and_Jewelry.json',nrows=10000,lines=True)
t

,_id,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,category,class
0,{'$oid': '5a132293741a2384e8376cbc'},A19PBP93OF896,0000031887,"Alinna Satake ""Can't Stop Eating""","[0, 1]",My 3-yr-old daughter received this as a gift f...,1,Tiny and Poorly Constructed!,1363824000,"03 21, 2013",Clothing_Shoes_and_Jewelry,0
1,{'$oid': '5a132293741a2384e8376cb9'},A2G0LNLN79Q6HR,0000031887,"aj_18 ""Aj_18""","[1, 1]",This was a really cute tutu the only problem i...,4,Really Cute but rather short.,1337990400,"05 26, 2012",Clothing_Shoes_and_Jewelry,1
2,{'$oid': '5a132293741a2384e8376cba'},A2XVJBSRI3SWDI,0000031887,abigail,"[0, 0]",Perfect red tutu for the price. I baught it as...,5,Nice tutu,1383523200,"11 4, 2013",Clothing_Shoes_and_Jewelry,1
3,{'$oid': '5a132293741a2384e8376cbd'},A1P0IHU93EF9ZK,0000031887,Amanda,"[0, 0]",Bought it for my daughters first birthday whic...,4,i love it,1390435200,"01 23, 2014",Clothing_Shoes_and_Jewelry,1
4,{'$oid': '5a132293741a2384e8376cbe'},A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","[0, 0]",This is a great tutu and at a really great pri...,5,Great tutu- not cheaply made,1297468800,"02 12, 2011",Clothing_Shoes_and_Jewelry,1
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,{'$oid': '5a132293741a2384e8379553'},A105HX4CY4GFS6,B00007GDAL,Karee,"[1, 1]","this has 16 card slots, one is a window thumb ...",5,Perfect for me,1373932800,"07 16, 2013",Clothing_Shoes_and_Jewelry,1
9996,{'$oid': '5a132293741a2384e8379560'},APAC4K0A9LBJJ,B00007GDAL,"Lainee,E","[0, 0]",I'm very satisfied with this wallet. Quality ...,5,Good Product,1396224000,"03 31, 2014",Clothing_Shoes_and_Jewelry,1
9997,{'$oid': '5a132293741a2384e837955f'},AC6C1VKYQBELX,B00007GDAL,Lady Sheep,"[1, 1]","I really like the wallet, I like that it is al...",5,great wallet,1378771200,"09 10, 2013",Clothing_Shoes_and_Jewelry,1
9998,{'$oid': '5a132293741a2384e837955d'},A17OBB1HKHY1KP,B00007GDAL,Kimberly M,"[0, 0]",I've been looking for something like this fore...,5,Finally,1402617600,"06 13, 2014",Clothing_Shoes_and_Jewelry,1


In [2]:
spark = SparkSession.builder.master("spark://localhost:7077").appName("BD").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/09 16:23:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [27]:
df = spark.read.json('Clothing_Shoes_and_Jewelry.json')
df = df.withColumnRenamed("summary","textSummary")
df.show()

+--------------------+----------+--------------------+-----+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|                 _id|      asin|            category|class|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|         textSummary|unixReviewTime|
+--------------------+----------+--------------------+-----+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|{5a132293741a2384...|0000031887|Clothing_Shoes_an...|  0.0| [0, 1]|    1.0|My 3-yr-old daugh...|03 21, 2013| A19PBP93OF896|Alinna Satake "Ca...|Tiny and Poorly C...|    1363824000|
|{5a132293741a2384...|0000031887|Clothing_Shoes_an...|  1.0| [1, 1]|    4.0|This was a really...|05 26, 2012|A2G0LNLN79Q6HR|       aj_18 "Aj_18"|Really Cute but r...|    1337990400|
|{5a132293741a2384...|0000031887|Clothing_Shoes_an...|  1.0| [0, 0]|    5.0|Perfect red tu

In [6]:
# TF-IDF

tokenizer = Tokenizer(inputCol="reviewText", outputCol="words")
wordsData = tokenizer.transform(df)

hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)


+-----+--------------------+
|class|            features|
+-----+--------------------+
|  0.0|(20,[0,1,2,3,4,5,...|
|  1.0|(20,[0,1,2,3,6,7,...|
|  1.0|(20,[0,3,4,6,8,9,...|
|  1.0|(20,[1,2,3,4,5,6,...|
|  1.0|(20,[0,1,2,3,4,5,...|
|  1.0|(20,[1,2,3,4,5,6,...|
|  0.0|(20,[0,1,3,4,5,6,...|
|  1.0|(20,[0,1,3,4,6,7,...|
|  1.0|(20,[0,1,2,3,4,5,...|
|  1.0|(20,[1,3,4,5,6,7,...|
|  1.0|(20,[1,2,3,4,5,6,...|
|  1.0|(20,[1,3,4,6,7,8,...|
|  1.0|(20,[0,2,3,6,8,9,...|
|  1.0|(20,[0,1,2,3,4,5,...|
|  1.0|(20,[0,1,2,3,4,5,...|
|  1.0|(20,[2,3,4,5,6,8,...|
|  0.0|(20,[1,2,3,4,5,6,...|
|  1.0|(20,[0,3,4,6,8,9,...|
|  1.0|(20,[0,1,3,4,5,6,...|
|  1.0|(20,[0,1,2,3,4,5,...|
+-----+--------------------+
only showing top 20 rows



In [8]:
rescaledData.show()

+--------------------+----------+--------------------+-----+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+
|                 _id|      asin|            category|class|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|               words|         rawFeatures|            features|
+--------------------+----------+--------------------+-----+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+
|{5a132293741a2384...|0000031887|Clothing_Shoes_an...|  0.0| [0, 1]|    1.0|My 3-yr-old daugh...|03 21, 2013| A19PBP93OF896|Alinna Satake "Ca...|Tiny and Poorly C...|    1363824000|[my, 3-yr-old, da...|(20,[0,1,2,3,4,5,...|(20,[0,1,2,3,4,5,...|
|{5a132293741a2384..

In [12]:
# Word2Vec

# Input data: Each row is a bag of words from a sentence or document.
l = df.select('class',split(df.reviewText, ' ').alias('str')).take(10000)
documentDF = spark.createDataFrame(data=l, schema = ['class','str'])
documentDF.show()

TypeError: Invalid argument, not a string or column: <bound method DataFrame.summary of DataFrame[_id: struct<$oid:string>, asin: string, category: string, class: double, helpful: array<bigint>, overall: double, reviewText: string, reviewTime: string, reviewerID: string, reviewerName: string, summary: string, unixReviewTime: bigint]> of type <class 'method'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [28]:
# Input data: Each row is a bag of words from a sentence or document.
documentDF = df.select('class',split(df.textSummary, ' ').alias('str'))
documentDF.show()

+-----+--------------------+
|class|                 str|
+-----+--------------------+
|  0.0|[Tiny, and, Poorl...|
|  1.0|[Really, Cute, bu...|
|  1.0|        [Nice, tutu]|
|  1.0|       [i, love, it]|
|  1.0|[Great, tutu-, , ...|
|  1.0|              [Tutu]|
|  0.0|[not, very, good,...|
|  1.0|        [Fantastic!]|
|  1.0|      [Very, Cute!!]|
|  1.0|        [Very, Nice]|
|  1.0|    [Good, Quality!]|
|  1.0|              [cute]|
|  1.0|     [super, cute!!]|
|  1.0|[My, daughter, lo...|
|  1.0|     [Good, product]|
|  1.0|        [Wonderful!]|
|  0.0|         [Waistband]|
|  1.0|            [Tutus!]|
|  1.0|      [Very, happy!]|
|  1.0|[Perfect, for, th...|
+-----+--------------------+
only showing top 20 rows



In [29]:
# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=7, minCount=0, inputCol="str", outputCol="result")
model = word2Vec.fit(documentDF)

result = model.transform(documentDF)

In [30]:
result.show()

+-----+--------------------+--------------------+
|class|                 str|              result|
+-----+--------------------+--------------------+
|  0.0|[Tiny, and, Poorl...|[-0.1789312288165...|
|  1.0|[Really, Cute, bu...|[-0.2102354392409...|
|  1.0|        [Nice, tutu]|[-0.1598797589540...|
|  1.0|       [i, love, it]|[0.89725091060002...|
|  1.0|[Great, tutu-, , ...|[-0.1689084346095...|
|  1.0|              [Tutu]|[0.02656632661819...|
|  0.0|[not, very, good,...|[-0.3106276933103...|
|  1.0|        [Fantastic!]|[0.07304732501506...|
|  1.0|      [Very, Cute!!]|[-0.7205476164817...|
|  1.0|        [Very, Nice]|[-0.6041751354932...|
|  1.0|    [Good, Quality!]|[-0.7159929871559...|
|  1.0|              [cute]|[-0.5383317470550...|
|  1.0|     [super, cute!!]|[-0.4520824253559...|
|  1.0|[My, daughter, lo...|[1.40153291821479...|
|  1.0|     [Good, product]|[-0.4305349886417...|
|  1.0|        [Wonderful!]|[0.10528587549924...|
|  0.0|         [Waistband]|[0.10752370953559...|


In [31]:
#Decision Tree

In [32]:
#Split data to train and test, 80% is for training and the rest is for tesing.
train, test = df.randomSplit([0.8, 0.2])

In [33]:
#Define decision tree and its hyper parameters for grid searching.
dt = DecisionTreeClassifier(featuresCol= 'features', labelCol = 'label')

dtParamGrid = ParamGridBuilder() \
    .addGrid(dt.minInstancesPerNode, [3, 5, 10, 15]) \
    .addGrid(dt.maxDepth, [3, 5, 10, 20]) \
    .addGrid(dt.maxBins, [8, 16, 32])\
    .build()

In [ ]:
# Using grid search to find best hyper parameter. The evaluation strategy is k-fold cross validation, in our case,
# k equals 5. In other words, each hyper parameters set is evaluated by 5-fold cross validation on traing data.  
# The best model is tested on test data, results are following.

In [34]:
bestDT = trainModel(train, dt, dtParamGrid)

acc, confusion, precision, reall = evaluateModel(test, bestDT)

fig, ax = plt.subplots(1, 1)
sns.heatmap(confusion, annot= True, cmap="PuBu", ax = ax, fmt= '.0f')
ax.set_xlabel("Prediction")
ax.set_ylabel("Ground Truth")
ax.set_yticklabels(["No", "Yes"])
ax.set_xticklabels(["No", "Yes"])

plt.show()

NameError: name 'trainModel' is not defined